In [5]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta

Задание №1

In [6]:
KEYWORDS = ['python', 'парсинг', 'питон']
req = requests.get('https://habr.com/ru/all/')

In [7]:
soup = BeautifulSoup(req.text, 'html.parser')
posts = soup.find_all('article', class_='post')

In [153]:
articles_list = []
for post in posts:
    cont_is = 0
    post_lower = post.text.lower()
    if any([desired in post_lower for desired in KEYWORDS]):
#    for keyword in KEYWORDS:
#        post_lower = post.text.lower()
#        if keyword in post_lower and cont_is == 0:
#            cont_is += 1
            #получаем дату статьи и приводим к единому формату, так как на сайте используется сегодня и вчера всесто дат.
            post_date = post.find('span', class_="post__time")
            post_date = post_date.text
            if "сегодня" in post_date:
                post_date = str (datetime.now().date()) + " в " + post_date.split(' ')[-1]
            if "вчера" in post_date:
                post_date = str (datetime.now().date() - timedelta(days=1)) + " в " + post_date.split(' ')[-1]
            links_full = post.find('a', class_='post__title_link')
            #получаем заголовок
            short_text_art = links_full.text
            #получаем ссылку
            links = links_full.attrs.get('href')
            #по ссылке получаем полный текст статьи
            req_page = requests.get(links)
            soup_1 = BeautifulSoup(req_page.text, 'html.parser')
            full_text_art = soup_1.find('div', class_="post__text").text
            time.sleep(0.2) #задержка на случай если много статей, чтобы не заблокировал сайт.
            articles_list.append ([post_date, short_text_art, links, full_text_art]) #формируем список
    

In [154]:
#формируем и выводим датафрейм
articles_df  = pd.DataFrame(articles_list, columns = ['Дата', 'Заголовок', 'Ссылка', 'Текст_статьи'])
articles_df

,Дата,Заголовок,Ссылка,Текст_статьи
0,2020-11-01 в 03:54,Файловый менеджер на питоне в 430 строк для на...,https://habr.com/ru/post/525976/,Всем привет!\n\r\nЯ захотел обобщить свои знан...


Задание №2

In [156]:
import json
EMAIL = ['xxx@x.ru', 'yyy@y.com']
URL='https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data' #запрос передается на этот адрес
emails_ = {"emailAddresses":EMAIL} #готовим список эл. почт для передачи в post
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:82.0) Gecko/20100101 Firefox/82.0', 
    'Content-Type': 'application/json;charset=utf-8',
    #Обязательны к передаче, как минимум два параметра ниже, предыдущие два не проверял
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

In [157]:
req_1 = requests.post (URL, json=emails_, headers=headers) #передаем post получаем ответ

In [158]:
resp_=req_1.json() #переводим в json

In [159]:
violation_list = [] #создаем пустой список
for domens_, i in resp_ ['data'].items(): #проходимся по словарю дата, получаем источники утечки
    for mails_, j in i.items(): # так как на один источник утечки может быть несколько почт, то получаем почты по источникам
        for k in j: #проходимся по записям и получаем ID шники утечек, по ним из словаря breaches тянем информацию по времени и описанию утечки
            breachId_= str(k['breachId'])
            #формируем список с нужными значениями
            violation_list.append([mails_, resp_ ['breaches'][breachId_]['publishDate'], domens_, resp_ ['breaches'][breachId_]['description']])

In [160]:
#формируем из списка датафрейм
violation_df  = pd.DataFrame(violation_list, columns = ['Почта', 'Дата утечки', 'Источник утечки', 'Описание утечки'])
violation_df

,Почта,Дата утечки,Источник утечки,Описание утечки
0,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
2,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
4,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
5,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
6,yyy@y.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
7,yyy@y.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
8,yyy@y.com,2020-05-28T00:00:00Z,wishbone.io,"In January 2020, the online poll website Wishb..."
9,yyy@y.com,2017-03-24T00:00:00Z,youku.com,Youku is a large Chinese video content company...


Задание №3

In [180]:
URL_REQUEST = 'https://api.vk.com/method/wall.get'
TOKEN = ''
VERSION = '5.103'
SLEEP = 0.33

In [185]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'owner_id': -30159897,
    'count': 100
}

In [186]:
req_vk = requests.get(URL_REQUEST, params)
req_vk.text

'{"error":{"error_code":10,"error_msg":"Internal server error: could not get application","request_params":[{"key":"v","value":"5.103"},{"key":"owner_id","value":"-30159897"},{"key":"count","value":"100"},{"key":"method","value":"wall.get"},{"key":"oauth","value":"1"}]}}'

In [ ]:
Возникает ошибка "error_code":10,"error_msg":"Internal server error: could not get application" 
        из-за чего не возможно дальше ничего сделать